In [1]:
import pandas as pd
from pandas import to_datetime
import xarray as xray
import numpy as np

In [2]:
# ICOS
########################################################

def icos_data_read(data_file, species):

    print("Reading " + data_file)

    ### Find out how many header lines there are
    #nheader = 0
    #with open(data_file, "r") as f:
    #    for l in f:
    #        if l[2]:
    #            nheader += 1
    #        else:
    #            break

    # Read CSV file
    df =  pd.read_csv(data_file,
                      skiprows=10,
                      names=['siteid',
                           'year', 
                           'month', 
                           'day', 
                           'hour', 
                           'minute', 
                           'second', 
                           'value', 
                           'value_unc', 
                           'nvalue', 
                           'latitude', 
                           'longitude', 
                           'altitude', 
                           'elevation', 
                           'intake_height', 
                           'QCflag',
                           'scale'],
                      parse_dates = {"time": ["year", "month", "day", "hour", "minute", "second"]},
                      index_col='time',
                      dtype = {"day": int,
                               "month": int,
                               "year": int,
                               "hour": int,
                               "minute": int,
                               "value_unc": float,
                               "altitude": float,
                               "nvalue": int},
                      na_values = "-999.99")

    # Format time
    df.index = pd.to_datetime(df.index, format = "%Y %m %d %H %M %S")
    
    # Set timestamp to left label
    df["new_time"] = df.index - pd.Timedelta(seconds = 30)
    df = df.set_index("new_time", inplace=False, drop=True)

    # Label time index
    df.index.name = "time"

    # Remove duplicates
    df = df.reset_index().drop_duplicates(subset='time').set_index('time')
    
    # Rename columns
    #df.rename(columns = {species.lower(): species.upper(),
    #                     "Stdev": species.upper() + " variability",
    #                     "NbPoints": species.upper() + " number_of_observations"},
    #           inplace = True)

    # Convert to Dataset
    ds = xray.Dataset.from_dataframe(df.sort_index())

    return ds

In [13]:
def icos(site, network = "ICOS",
         input_directory = None,
         output_directory = None,
         date_range = None,
         version = None):

    def find_species_inlet_model(filenames):
        out = []
        for f in filenames:
            f_elements = f.split(".")
            if len(f_elements) == 6:
                out.append((f_elements[1],
                            f_elements[4],
                            "picarro" + f_elements[3].upper()))
            else:
                out.append((f_elements[1],
                            f_elements[3],
                            "picarro"))
        return(out)

    # Get directories and site strings
    params_icos = params["ICOS"]
    site_string = params_icos[site]["gcwerks_site_name"]

    data_folder, output_folder = \
            get_directories(params_icos["directory"].replace("%site", site_string),
                            params_icos["directory_output"],
                            user_specified_input_directory = input_directory,
                            user_specified_output_directory = output_directory)

    # Search for species, inlets and model from file names
    data_file_search = join(data_folder, species.lower() + site.upper() + "_*_verify.csv")
    data_files = glob.glob(data_file_search)
    data_file_names = [split(f)[1] for f in data_files]
    species_inlet_model = find_species_inlet_model(data_file_names)

    inlets = set([i for (s, i, m) in species_inlet_model])

    for i, (species, inlet, model) in enumerate(species_inlet_model):

        if stat(data_files[i]).st_size > 0:

            # Create xarray dataset
            ds = icos_data_read(data_files[i], species.upper())

            # Sort out attributes
            global_attributes = params_icos[site.upper()]["global_attributes"]
            global_attributes["inlet_height_magl"] = float(params_icos[site]["inlet_rename"][inlet][:-1])

            ds = attributes(ds,
                            species.upper(),
                            site.upper(),
                            network = network,
                            global_attributes = global_attributes,
                            sampling_period = 60,
                            date_range = date_range)

            if len(ds.time.values) == 0:

                # Then must have not passed date_range filter?
                print(" ... no data in range")
                # then do nothing

            else:

                inlet_label = params_icos[site]["inlet_rename"][inlet]

                # Write file
                nc_filename = output_filename(output_folder,
                                              network,
                                              model,
                                              site.upper(),
                                              ds.time.to_pandas().index.to_pydatetime()[0],
                                              ds.species,
                                              inlet = [None, inlet_label][len(inlets) > 1],
                                              version = version)

                ds.to_netcdf(nc_filename)

                print("Written " + nc_filename)

        else:
            print("Skipping empty file: %s" % data_files[i])


In [15]:
icos("bgu", network="ICOS")

NameError: name 'params' is not defined

In [12]:
data_folder = "core"
species = "ch4"
site="bgu"
data_file_search = data_folder.join(species.lower() + site.upper() + "_*_verify.csv")
data_file_search

'ccorehcore4coreBcoreGcoreUcore_core*core_corevcoreecorercoreicorefcoreycore.coreccorescorev'

In [3]:
def icos(site, network = "ICOS",
         input_directory = None,
         output_directory = None,
         date_range = None,
         version = None):

    def find_species_inlet_model(filenames):
        out = []
        for f in filenames:
            f_elements = f.split(".")
            if len(f_elements) == 6:
                out.append((f_elements[1],
                            f_elements[4],
                            "picarro" + f_elements[3].upper()))
            else:
                out.append((f_elements[1],
                            f_elements[3],
                            "picarro"))
        return(out)

    # Get directories and site strings
    params_icos = params["ICOS"]
    site_string = params_icos[site]["gcwerks_site_name"]

    data_folder, output_folder = \
            get_directories(params_icos["directory"].replace("%site", site_string),
                            params_icos["directory_output"],
                            user_specified_input_directory = input_directory,
                            user_specified_output_directory = output_directory)

    # Search for species, inlets and model from file names
    data_file_search = join(data_folder, site.lower() + ".*.1minute.*.dat")
    data_files = glob.glob(data_file_search)
    data_file_names = [split(f)[1] for f in data_files]
    species_inlet_model = find_species_inlet_model(data_file_names)

    inlets = set([i for (s, i, m) in species_inlet_model])

    for i, (species, inlet, model) in enumerate(species_inlet_model):

        if stat(data_files[i]).st_size > 0:

            # Create xarray dataset
            ds = icos_data_read(data_files[i], species.upper())

            # Sort out attributes
            global_attributes = params_icos[site.upper()]["global_attributes"]
            global_attributes["inlet_height_magl"] = float(params_icos[site]["inlet_rename"][inlet][:-1])

            ds = attributes(ds,
                            species.upper(),
                            site.upper(),
                            network = network,
                            global_attributes = global_attributes,
                            sampling_period = 60,
                            date_range = date_range)

            if len(ds.time.values) == 0:

                # Then must have not passed date_range filter?
                print(" ... no data in range")
                # then do nothing

            else:

                inlet_label = params_icos[site]["inlet_rename"][inlet]

                # Write file
                nc_filename = output_filename(output_folder,
                                              network,
                                              model,
                                              site.upper(),
                                              ds.time.to_pandas().index.to_pydatetime()[0],
                                              ds.species,
                                              inlet = [None, inlet_label][len(inlets) > 1],
                                              version = version)

                ds.to_netcdf(nc_filename)

                print("Written " + nc_filename)

        else:
            print("Skipping empty file: %s" % data_files[i])


In [4]:
icos = icos_data_read(data_file="core/ch4_bgu_lsceflask_verify.csv", species="ch4")
icos

Reading core/ch4_bgu_lsceflask_verify.csv


<xarray.Dataset>
Dimensions:        (time: 439)
Coordinates:
  * time           (time) datetime64[ns] 2005-01-04T12:04:30 ... 2018-01-03T1...
Data variables:
    siteid         (time) object 'BGU' 'BGU' 'BGU' 'BGU' ... 'BGU' 'BGU' 'BGU'
    value          (time) float64 1.951e+03 1.917e+03 ... 1.95e+03 -1e+03
    value_unc      (time) float64 -1e+03 -1e+03 -1e+03 ... -1e+03 -1e+03 -1e+03
    nvalue         (time) int64 4 4 4 4 4 2 4 4 4 4 4 ... 4 4 4 4 4 4 4 4 4 4 4
    latitude       (time) float64 41.97 41.97 41.97 41.97 ... 41.97 41.97 41.97
    longitude      (time) float64 3.23 3.23 3.23 3.23 ... 3.23 3.23 3.23 3.23
    altitude       (time) float64 15.0 15.0 15.0 15.0 ... 15.0 15.0 15.0 15.0
    elevation      (time) int64 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13
    intake_height  (time) int64 2 2 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2 2 2
    QCflag         (time) int64 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 3
    scale          (time) int64 3 3 3 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3 3

In [105]:
    nheader = 0
    with open("core/ch4_bgu_lsceflask_verify.csv", "r") as f:
        for l in f:
            if l[0] == "units : ppb":
                nheader += 1
            break
                
    print(nheader)

0


In [108]:
DECC = xray.open_dataset("DECC-picarro_RGL_20120223_ch4-90m-20220929.nc")
DECC

<xarray.Dataset>
Dimensions:                     (time: 3208934)
Coordinates:
  * time                        (time) datetime64[ns] 2012-02-23T16:00:00 ......
Data variables:
    ch4                         (time) float32 ...
    ch4_variability             (time) float32 ...
    ch4_number_of_observations  (time) int16 ...
Attributes: (12/15)
    data_owner:           Simon O'Doherty
    data_owner_email:     s.odoherty@bristol.ac.uk
    inlet_height_magl:    90.0
    comment:              Cavity ring-down measurements. Output from GCWerks
    Conditions of use:    Ensure that you contact the data owner at the outse...
    Source:               In situ measurements of air
    ...                   ...
    species:              ch4
    Calibration_scale:    WMO-X2004A
    station_longitude:    -2.5399
    station_latitude:     51.997435
    station_long_name:    Ridge Hill, UK
    station_height_masl:  200.0

In [ ]:
siteid year month day hour minute second value value_unc nvalue latitude longitude altitude elevation intake_height QCflag scale